In [1]:
%cd ..

/Users/mateoibarguen/Desktop/CSC 482/genealogy-extraction


In [2]:
from src.relation_models.relation_models import *
from src.wiki_referencer.wiki_reference import WikiReferencer
from itertools import combinations 

In [31]:
article_id = '2204744'

In [7]:
model = EntityFeatureRelationModel(num_train=50)
model.fit_train()

here
sibling her elder brother, @Q2800847@
here
sibling his brother @Q551463@


In [8]:
model.classifier.show_most_informative_features()

Most Informative Features
first_occurrence_entities_in_same_sentence = True              P26 : P40    =      7.5 : 1.0
first_occurrence_entities_in_between = 1                 P26 : P3373  =      5.4 : 1.0
shortest_occurrence_entities_in_same_sentence = True              P26 : P3373  =      3.9 : 1.0
shortest_occurrence_entities_in_between = 0                 P26 : P3373  =      3.2 : 1.0
shortest_occurrence_words_in_between = 2                 P26 : P3373  =      2.7 : 1.0
first_occurrence_words_in_between = None            P3373 : P25    =      2.4 : 1.0
shortest_occurrence_words_in_between = None            P3373 : P25    =      2.3 : 1.0
shortest_occurrence_entities_in_between = 1                 P25 : P3373  =      2.1 : 1.0
first_occurrence_entities_in_between = 0                 P40 : P3373  =      1.9 : 1.0
shortest_occurrence_entities_in_between = None            P3373 : P25    =      1.8 : 1.0


In [9]:
model.evaluate_test()

here
child her eldest daughter @Q61928134@
here
sibling their brother @Q737820@
test accuracy:  0.17647058823529413


0.17647058823529413

In [50]:
def get_most_prob(list_comb_probs):
    max_class = [(p[0], p[1], p[1].max()) for p in list_comb_probs]
    max_prob = [(c[0], c[2], c[1].prob(c[2])) for c in max_class]
    return sorted(max_prob, key=lambda x: x[2], reverse=True)

In [64]:
wiki_referencer = WikiReferencer()

def predict_relations_article(article_id):
    entities_article = wiki_referencer.get_article_entities(article_id)
    entities_combinations = list(combinations(entities_article, 2))
    entities_probs = [model.predict_relation_from_ids(article_id=article_id, entity_a_id=e_a, entity_b_id=e_b) for e_a, e_b in entities_combinations]
    entities_probs = [(entities_combinations[i], entities_probs[i]) for i in range(len(entities_combinations))]
    return entities_probs
    

In [42]:
#comb_probs = predict_relations_article(article_id)

P26: "spouse"
P40: "child"
P25: "mother"
P22: "father"
P3373: "sibling"

In [84]:
def get_most_prob(self, list_comb_probs):
    max_class = [(p[0], p[1], p[1].max()) for p in list_comb_probs]
    max_prob = [(c[0], c[2], c[1].prob(c[2])) for c in max_class]
    return sorted(max_prob, key=lambda x: x[2], reverse=True)

def predict_relations_article(article_id, wiki_referencer, model):
    article_entities = wiki_referencer.get_article_entities(article_id)
    entities_combinations = list(combinations(article_entities, 2))
    entities_probs = [model.predict_relation_from_ids(article_id=article_id, entity_a_id=e_a, entity_b_id=e_b) for e_a, e_b in entities_combinations]
    entities_probs = [(entities_combinations[i], entities_probs[i]) for i in range(len(entities_combinations))]
    return entities_probs

In [85]:
entity_probs = predict_relations_article(article_id, wiki_referencer, model)
entity_probs

[(('Q13133', 'Q15070044'), <ProbDist with 4 samples>),
 (('Q13133', 'Q15070048'), <ProbDist with 4 samples>),
 (('Q13133', 'Q15982139'), <ProbDist with 4 samples>),
 (('Q13133', 'Q76'), <ProbDist with 4 samples>),
 (('Q13133', 'Q6761976'), <ProbDist with 4 samples>),
 (('Q13133', 'Q2073960'), <ProbDist with 4 samples>),
 (('Q15070044', 'Q15070048'), <ProbDist with 4 samples>),
 (('Q15070044', 'Q15982139'), <ProbDist with 4 samples>),
 (('Q15070044', 'Q76'), <ProbDist with 4 samples>),
 (('Q15070044', 'Q6761976'), <ProbDist with 4 samples>),
 (('Q15070044', 'Q2073960'), <ProbDist with 4 samples>),
 (('Q15070048', 'Q15982139'), <ProbDist with 4 samples>),
 (('Q15070048', 'Q76'), <ProbDist with 4 samples>),
 (('Q15070048', 'Q6761976'), <ProbDist with 4 samples>),
 (('Q15070048', 'Q2073960'), <ProbDist with 4 samples>),
 (('Q15982139', 'Q76'), <ProbDist with 4 samples>),
 (('Q15982139', 'Q6761976'), <ProbDist with 4 samples>),
 (('Q15982139', 'Q2073960'), <ProbDist with 4 samples>),
 (('Q7

In [131]:
ex1 = {'P26', 'P22'}
ex2 = {'P26', 'a', 'b'}
ex2 - (ex1 - ex2)

{'P26', 'a', 'b'}

In [104]:
prob.prob('P26')

0.7885076580728498

In [111]:
possible_relations = list({'P26', 'P22', 'P40'})
possible_relations_prob = [(relation, prob.prob(relation)) for relation in possible_relations]

In [122]:
import numpy as np

In [125]:
max(possible_relations_prob, key=lambda x: x[1])

('P26', 0.7885076580728498)

In [207]:
class ArticleTree:
    def __init__(self, article_id, wiki_referencer, entities_probs):
        self.article_id = article_id
        self.wiki_referencer = wiki_referencer
        self.model = model
        self.article_entities = self.wiki_referencer.get_article_entities(self.article_id)
        self.family_tree = self.initialize_tree(self.article_id)
        self.entities_probs = entities_probs
        self.entities_probs_dict = {comb[0]: comb[1] for comb in self.entities_probs}
        self.relations = ['P26', 'P25', 'P40', 'P22', 'P3373']
        self.relation_maps = {'P22': self.add_father, 
                        'P26': self.add_spouse, 
                        'P25': self.add_mother, 'P3373': self.add_sibling,
                        'P40': self.add_child}
        self.most_prob = self.initialize_most_prob()
        
    def initialize_most_prob(self):
        max_class = [(p[0], p[1], p[1].max()) for p in self.entities_probs]
        max_prob = [(c[0], c[2], c[1].prob(c[2])) for c in max_class]
        return sorted(max_prob, key=lambda x: x[2], reverse=True)
    
    def get_most_prob(self, comb):
        only_one_possible = {'P26', 'P22', '25'}
        possible_relations = self.get_possible_relations(comb[0])
        comb_prob = self.entities_probs_dict[comb]
        possible_relations_prob = [(relation, comb_prob.prob(relation)) for relation in possible_relations]
        print('possible_relations_prob: ', possible_relations_prob)
        return max(possible_relations_prob, key=lambda x: x[1])
        
    def get_possible_relations(self, entity):
        entity_relations = set(self.family_tree[entity].values())
        once_per_entity = {'P25', 'P22', 'P26'}
        many_per_entity = {'P40', 'P3373'}
        possible_relations = once_per_entity - entity_relations
        possible_relations.update(many_per_entity)
        return list(possible_relations)
    
    def updated_probs(self, combs):
        try:
            return max([(comb, self.get_most_prob(comb)) for comb in combs], key=lambda x: x[1][1])
        except:
            print('No relations. ')
            return None
        
        
    def initialize_tree(self, article_id):
        family_tree = {}
        for entity in self.article_entities:
            article_entities_c = self.article_entities.copy()
            article_entities_c.remove(entity)
            entity_tree = {entity: {ar_ent: None for ar_ent in article_entities_c}}
            family_tree.update(entity_tree)
        return family_tree
    
    def add_father(self, comb):
        # comb[0] and comb[1] can't have relation.
        # comb[0] can't have father. 

        if 'P22' not in set(self.family_tree[comb[0]].values()):
            if self.family_tree[comb[0]][comb[1]] is None and self.family_tree[comb[1]][comb[0]] is None:
                self.family_tree[comb[0]][comb[1]] = 'P22'
                self.family_tree[comb[1]][comb[0]] = 'P40'
    
    def add_mother(self, comb):
        # comb[0] and comb[1] can't have relation.
        # comb[0] can't have father. 

        if 'P25' not in set(self.family_tree[comb[0]].values()):
            if self.family_tree[comb[0]][comb[1]] is None and self.family_tree[comb[1]][comb[0]] is None:
                self.family_tree[comb[0]][comb[1]] = 'P25'
                self.family_tree[comb[1]][comb[0]] = 'P40'
                
    def add_spouse(self, comb):
        # comb[0] and comb[1] can't have relation.
        # comb[0] can't have father. 

        if 'P26' not in set(self.family_tree[comb[0]].values()):
            if self.family_tree[comb[0]][comb[1]] is None and self.family_tree[comb[1]][comb[0]] is None:
                self.family_tree[comb[0]][comb[1]] = 'P26'
                self.family_tree[comb[1]][comb[0]] = 'P26'
                
                
    def add_sibling(self, comb):
        if self.family_tree[comb[0]][comb[1]] is None and self.family_tree[comb[1]][comb[0]] is None:
                self.family_tree[comb[0]][comb[1]] = 'P3373'
                self.family_tree[comb[1]][comb[0]] = 'P3373'
                
    def add_child(self, comb):
        if self.family_tree[comb[0]][comb[1]] is None and self.family_tree[comb[1]][comb[0]] is None:
                self.family_tree[comb[0]][comb[1]] = 'P40'
                
    def add_relation(self, comb, relation):
        self.relation_maps[relation](comb)
        
    def get_relations(self):
        combos = list(self.entities_probs_dict.keys())
        added_relations = []
        most_prob_comb = self.most_prob[0]
        print(most_prob_comb)
        while len(combos) > 0:
            
            relation, relation_prob = self.get_most_prob(most_prob_comb[0])
            print('relation: ', relation)
            self.add_relation(most_prob_comb[0], relation)
            
            added_relations.append((most_prob_comb[0], relation))
            combos.remove(most_prob_comb[0])
            most_prob_comb = self.updated_probs(combos)
            
        return added_relations
    

In [208]:
article_tree = ArticleTree(article_id, wiki_referencer, entity_probs)

In [209]:
article_tree.entities_probs_dict[('Q13133', 'Q15070044')].prob('P26')

0.4796025804849383

In [210]:
article_tree.get_relations()

(('Q13133', 'Q6761976'), 'P26', 0.8138087823840999)
possible_relations_prob:  [('P3373', 0.00897953489627304), ('P25', 0.16501021567749885), ('P26', 0.8138087823840999), ('P22', 0), ('P40', 0.012201467042128471)]
relation:  P26
possible_relations_prob:  [('P22', 0), ('P3373', 0.0440993040075771), ('P25', 0.2917368070671151), ('P40', 0.18456130844036867)]
possible_relations_prob:  [('P22', 0), ('P3373', 0.06098404374374332), ('P25', 0.5586050547229175), ('P40', 0.2103512099646012)]
possible_relations_prob:  [('P22', 0), ('P3373', 0.09717076352974328), ('P25', 0.2966900648475987), ('P40', 0.33516943821493783)]
possible_relations_prob:  [('P22', 0), ('P3373', 0.011463505099215805), ('P25', 0.16431190113297756), ('P40', 0.013859807301563333)]
possible_relations_prob:  [('P22', 0), ('P3373', 0.09717076352974328), ('P25', 0.2966900648475987), ('P40', 0.33516943821493783)]
possible_relations_prob:  [('P3373', 0.011463505099215834), ('P25', 0.16431190113297756), ('P26', 0.8103647864662431), ('

[(('Q13133', 'Q6761976'), 'P26'),
 (('Q15070044', 'Q15070048'), 'P26'),
 (('Q15982139', 'Q6761976'), 'P26'),
 (('Q76', 'Q6761976'), 'P26'),
 (('Q13133', 'Q15070048'), 'P25'),
 (('Q15982139', 'Q76'), 'P25'),
 (('Q15070044', 'Q15982139'), 'P25'),
 (('Q15070044', 'Q2073960'), 'P40'),
 (('Q15070048', 'Q2073960'), 'P40'),
 (('Q15070048', 'Q76'), 'P40'),
 (('Q13133', 'Q15982139'), 'P40'),
 (('Q13133', 'Q2073960'), 'P40'),
 (('Q15070048', 'Q15982139'), 'P40'),
 (('Q15982139', 'Q2073960'), 'P40'),
 (('Q6761976', 'Q2073960'), 'P40'),
 (('Q76', 'Q2073960'), 'P40'),
 (('Q13133', 'Q15070044'), 'P40'),
 (('Q15070048', 'Q6761976'), 'P25'),
 (('Q15070044', 'Q6761976'), 'P3373'),
 (('Q13133', 'Q76'), 'P40'),
 (('Q15070044', 'Q76'), 'P3373')]

In [194]:
article_tree.most_prob

[(('Q13133', 'Q6761976'), 'P26', 0.8138087823840999),
 (('Q13133', 'Q76'), 'P26', 0.8103647864662431),
 (('Q15070044', 'Q15070048'), 'P26', 0.8103647864662431),
 (('Q15070048', 'Q6761976'), 'P26', 0.7885076580728498),
 (('Q15070044', 'Q76'), 'P26', 0.7309998543102055),
 (('Q15982139', 'Q6761976'), 'P26', 0.7239099217885213),
 (('Q15070044', 'Q6761976'), 'P26', 0.7120328003856331),
 (('Q76', 'Q6761976'), 'P26', 0.7064934129623964),
 (('Q13133', 'Q15070048'), 'P25', 0.5586050547229175),
 (('Q15982139', 'Q76'), 'P25', 0.5586050547229175),
 (('Q76', 'Q2073960'), 'P26', 0.5222168556091572),
 (('Q15070044', 'Q15982139'), 'P25', 0.4900636646406334),
 (('Q13133', 'Q15070044'), 'P26', 0.4796025804849383),
 (('Q15070044', 'Q2073960'), 'P40', 0.42647058823529405),
 (('Q15070048', 'Q2073960'), 'P40', 0.42647058823529405),
 (('Q15070048', 'Q76'), 'P40', 0.4044078437492772),
 (('Q13133', 'Q15982139'), 'P40', 0.33516943821493783),
 (('Q13133', 'Q2073960'), 'P40', 0.33516943821493783),
 (('Q15070048',

In [211]:
article_tree.get_relations()

(('Q13133', 'Q6761976'), 'P26', 0.8138087823840999)
possible_relations_prob:  [('P22', 0), ('P3373', 0.00897953489627304), ('P40', 0.012201467042128471)]
relation:  P40
possible_relations_prob:  [('P22', 0), ('P3373', 0.0440993040075771), ('P40', 0.18456130844036867)]
possible_relations_prob:  [('P22', 0), ('P3373', 0.06098404374374332), ('P40', 0.2103512099646012)]
possible_relations_prob:  [('P22', 0), ('P3373', 0.09717076352974328), ('P40', 0.33516943821493783)]
possible_relations_prob:  [('P22', 0), ('P3373', 0.011463505099215805), ('P40', 0.013859807301563333)]
possible_relations_prob:  [('P22', 0), ('P3373', 0.09717076352974328), ('P40', 0.33516943821493783)]
possible_relations_prob:  [('P22', 0), ('P3373', 0.011463505099215834), ('P40', 0.013859807301563333)]
possible_relations_prob:  [('P22', 0), ('P3373', 0.05778134086990445), ('P40', 0.004575394448947611)]
possible_relations_prob:  [('P22', 0), ('P3373', 0.013443040599208216), ('P40', 0.00852437407938347)]
possible_relations_

[(('Q13133', 'Q6761976'), 'P40'),
 (('Q15070044', 'Q2073960'), 'P40'),
 (('Q15070048', 'Q2073960'), 'P40'),
 (('Q15070048', 'Q76'), 'P40'),
 (('Q13133', 'Q15982139'), 'P40'),
 (('Q13133', 'Q2073960'), 'P40'),
 (('Q15070048', 'Q15982139'), 'P40'),
 (('Q15982139', 'Q2073960'), 'P40'),
 (('Q6761976', 'Q2073960'), 'P40'),
 (('Q76', 'Q2073960'), 'P40'),
 (('Q76', 'Q6761976'), 'P25'),
 (('Q13133', 'Q15070048'), 'P40'),
 (('Q15982139', 'Q76'), 'P40'),
 (('Q13133', 'Q15070044'), 'P40'),
 (('Q15982139', 'Q6761976'), 'P3373'),
 (('Q15070044', 'Q15982139'), 'P3373'),
 (('Q15070048', 'Q6761976'), 'P40'),
 (('Q15070044', 'Q6761976'), 'P3373'),
 (('Q13133', 'Q76'), 'P40'),
 (('Q15070044', 'Q15070048'), 'P40'),
 (('Q15070044', 'Q76'), 'P3373')]

In [161]:
article_tree.add_comb()

(('Q15070048', 'Q6761976'), ('P26', 0.7885076580728498))


In [140]:
article_tree.get_most_prob(('Q13133', 'Q6761976'))

('P26', 0.8138087823840999)

In [141]:
article_tree.add_spouse(('Q13133', 'Q6761976'))

In [142]:
article_tree.get_most_prob(('Q13133', 'Q6761976'))

('P25', 0.16501021567749885)

In [115]:
article_tree.most_prob

[(('Q13133', 'Q6761976'), 'P26', 0.8138087823840999),
 (('Q13133', 'Q76'), 'P26', 0.8103647864662431),
 (('Q15070044', 'Q15070048'), 'P26', 0.8103647864662431),
 (('Q15070048', 'Q6761976'), 'P26', 0.7885076580728498),
 (('Q15070044', 'Q76'), 'P26', 0.7309998543102055),
 (('Q15982139', 'Q6761976'), 'P26', 0.7239099217885213),
 (('Q15070044', 'Q6761976'), 'P26', 0.7120328003856331),
 (('Q76', 'Q6761976'), 'P26', 0.7064934129623964),
 (('Q13133', 'Q15070048'), 'P25', 0.5586050547229175),
 (('Q15982139', 'Q76'), 'P25', 0.5586050547229175),
 (('Q76', 'Q2073960'), 'P26', 0.5222168556091572),
 (('Q15070044', 'Q15982139'), 'P25', 0.4900636646406334),
 (('Q13133', 'Q15070044'), 'P26', 0.4796025804849383),
 (('Q15070044', 'Q2073960'), 'P40', 0.42647058823529405),
 (('Q15070048', 'Q2073960'), 'P40', 0.42647058823529405),
 (('Q15070048', 'Q76'), 'P40', 0.4044078437492772),
 (('Q13133', 'Q15982139'), 'P40', 0.33516943821493783),
 (('Q13133', 'Q2073960'), 'P40', 0.33516943821493783),
 (('Q15070048',

In [109]:
# Q13133: Michelle Obama
# Q76: Barack Obama
wiki_referencer.relations['Q76']['Q15070048']

'P40'

In [97]:
article_tree.get_most_prob()

[(('Q13133', 'Q6761976'), 'P26', 0.8138087823840999),
 (('Q13133', 'Q76'), 'P26', 0.8103647864662431),
 (('Q15070044', 'Q15070048'), 'P26', 0.8103647864662431),
 (('Q15070048', 'Q6761976'), 'P26', 0.7885076580728498),
 (('Q15070044', 'Q76'), 'P26', 0.7309998543102055),
 (('Q15982139', 'Q6761976'), 'P26', 0.7239099217885213),
 (('Q15070044', 'Q6761976'), 'P26', 0.7120328003856331),
 (('Q76', 'Q6761976'), 'P26', 0.7064934129623964),
 (('Q13133', 'Q15070048'), 'P25', 0.5586050547229175),
 (('Q15982139', 'Q76'), 'P25', 0.5586050547229175),
 (('Q76', 'Q2073960'), 'P26', 0.5222168556091572),
 (('Q15070044', 'Q15982139'), 'P25', 0.4900636646406334),
 (('Q13133', 'Q15070044'), 'P26', 0.4796025804849383),
 (('Q15070044', 'Q2073960'), 'P40', 0.42647058823529405),
 (('Q15070048', 'Q2073960'), 'P40', 0.42647058823529405),
 (('Q15070048', 'Q76'), 'P40', 0.4044078437492772),
 (('Q13133', 'Q15982139'), 'P40', 0.33516943821493783),
 (('Q13133', 'Q2073960'), 'P40', 0.33516943821493783),
 (('Q15070048',

In [71]:
def initialize_tree(article_id):
    family_tree = {}
    article_entities = wiki_referencer.get_article_entities(article_id=article_id)
    for entity in article_entities:
        article_entities_c = article_entities.copy()
        article_entities_c.remove(entity)
        entity_tree = {entity: {ar_ent: None for ar_ent in article_entities_c}}
        family_tree.update(entity_tree)
    return family_tree

In [72]:
family_tree = initialize_tree(article_id)

In [81]:
def add_father(comb):
    # comb[0] and comb[1] can't have relation.
    # comb[0] can't have father. 
    
    if 'P26' not in list(family_tree[comb[0]].values()):
        if family_tree[comb[0]][comb[1]] is None and family_tree[comb[1]][comb[0]] is None:
            family_tree[comb[0]][comb[1]] = 'P26'
            family_tree[comb[1]][comb[0]] = 'P40'
    return family_tree

In [ ]:
def add_mother(comb):
    # comb[0] and comb[1] can't have relation.
    # comb[0] can't have father. 
    
    if 'P25' not in list(family_tree[comb[0]].values()):
        if family_tree[comb[0]][comb[1]] is None and family_tree[comb[1]][comb[0]] is None:
            family_tree[comb[0]][comb[1]] = 'P25'
            family_tree[comb[1]][comb[0]] = 'P40'
    return family_tree

In [82]:
add_father(('Q13133', 'Q15070044'))

{'Q13133': {'Q15070044': 'P26',
  'Q15070048': None,
  'Q15982139': None,
  'Q76': None,
  'Q6761976': None,
  'Q2073960': None},
 'Q15070044': {'Q13133': 'P40',
  'Q15070048': None,
  'Q15982139': None,
  'Q76': None,
  'Q6761976': None,
  'Q2073960': None},
 'Q15070048': {'Q13133': None,
  'Q15070044': None,
  'Q15982139': None,
  'Q76': None,
  'Q6761976': None,
  'Q2073960': None},
 'Q15982139': {'Q13133': None,
  'Q15070044': None,
  'Q15070048': None,
  'Q76': None,
  'Q6761976': None,
  'Q2073960': None},
 'Q76': {'Q13133': None,
  'Q15070044': None,
  'Q15070048': None,
  'Q15982139': None,
  'Q6761976': None,
  'Q2073960': None},
 'Q6761976': {'Q13133': None,
  'Q15070044': None,
  'Q15070048': None,
  'Q15982139': None,
  'Q76': None,
  'Q2073960': None},
 'Q2073960': {'Q13133': None,
  'Q15070044': None,
  'Q15070048': None,
  'Q15982139': None,
  'Q76': None,
  'Q6761976': None}}

In [80]:
family_tree['Q13133'].values()

dict_values([None, None, None, None, None, None])

In [73]:
initialize_tree(article_id)

{'Q13133': {'Q15070044': None,
  'Q15070048': None,
  'Q15982139': None,
  'Q76': None,
  'Q6761976': None,
  'Q2073960': None},
 'Q15070044': {'Q13133': None,
  'Q15070048': None,
  'Q15982139': None,
  'Q76': None,
  'Q6761976': None,
  'Q2073960': None},
 'Q15070048': {'Q13133': None,
  'Q15070044': None,
  'Q15982139': None,
  'Q76': None,
  'Q6761976': None,
  'Q2073960': None},
 'Q15982139': {'Q13133': None,
  'Q15070044': None,
  'Q15070048': None,
  'Q76': None,
  'Q6761976': None,
  'Q2073960': None},
 'Q76': {'Q13133': None,
  'Q15070044': None,
  'Q15070048': None,
  'Q15982139': None,
  'Q6761976': None,
  'Q2073960': None},
 'Q6761976': {'Q13133': None,
  'Q15070044': None,
  'Q15070048': None,
  'Q15982139': None,
  'Q76': None,
  'Q2073960': None},
 'Q2073960': {'Q13133': None,
  'Q15070044': None,
  'Q15070048': None,
  'Q15982139': None,
  'Q76': None,
  'Q6761976': None}}